In [1]:
%load_ext autoreload
%autoreload 2 
%reload_ext autoreload
%matplotlib inline

import numpy as np
import scipy.io as io
from pyDOE import lhs
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

from complexPyTorch.complexLayers import ComplexLinear

import cplxmodule
from cplxmodule import cplx
from cplxmodule.nn import RealToCplx, CplxToReal, CplxSequential, CplxToCplx
from cplxmodule.nn import CplxLinear, CplxModReLU, CplxAdaptiveModReLU, CplxModulus, CplxAngle

# To access the contents of the parent dir
import sys; sys.path.insert(0, '../')
import os
from scipy.io import loadmat
from utils import *
from models import TorchComplexMLP, ImaginaryDimensionAdder, cplx2tensor, ComplexTorchMLP, complex_mse
from preprocess import *

# Model selection
from sparsereg.model import STRidge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge
from pde_diff import TrainSTRidge, FiniteDiff, print_pde
from RegscorePy.bic import bic

from madgrad import MADGRAD

/usr/local/lib/python3.9/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
# torch device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("You're running on", device)

DATA_PATH = '../PDE_FIND_experimental_datasets/harmonic_osc.mat'
data = io.loadmat(DATA_PATH)

t = data['t'].flatten()[:,None]
x = data['x'].flatten()[:,None]

spatial_dim = x.shape[0]
time_dim = t.shape[0]

potential = np.vstack([0.5*np.power(x,2).reshape((1,spatial_dim)) for _ in range(time_dim)])
Exact = data['usol']

X, T = np.meshgrid(x, t)

# Adjust the diemnsion of Exact and potential (0.5*x**2)
if Exact.T.shape == X.shape: Exact = Exact.T
if potential.T.shape == X.shape: potential = potential.T
Exact_u = np.real(Exact)
Exact_v = np.imag(Exact)

X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
h_star = to_column_vector(Exact)
u_star = to_column_vector(Exact_u)
v_star = to_column_vector(Exact_v)

# Doman bounds
lb = X_star.min(axis=0)
ub = X_star.max(axis=0)

# Converting the grounds to be tensor
X_star = to_tensor(X_star, True)
h_star = to_complex_tensor(h_star, False)

N = 500; include_N_res = 2
idx = np.random.choice(X_star.shape[0], N, replace=False)
# idx = np.arange(N) # Just have an easy dataset for experimenting

lb = to_tensor(lb, False).to(device)
ub = to_tensor(ub, False).to(device)

X_train = to_tensor(X_star[idx, :], True).to(device)
u_train = to_tensor(u_star[idx, :], False).to(device)
v_train = to_tensor(v_star[idx, :], False).to(device)
h_train = torch.complex(u_train, v_train).to(device)

# Unsup data
if include_N_res>0:
    N_res = int(N*include_N_res)
    idx_res = np.array(range(X_star.shape[0]-1))[~idx]
    idx_res = idx_res[:N_res]
    X_res = to_tensor(X_star[idx_res, :], True)
    print(f"Training with {N_res} unsup samples")
    X_train = torch.vstack([X_train, X_res])

# Potential is calculated from x
# Hence, Quadratic features of x are required.
feature_names = ['hf', 'x', 'h_x', 'h_xx', 'h_xxx']

You're running on cpu
Training with 1000 unsup samples


/Users/pongpisit/Desktop/Multi-task-Physics-informed-neural-networks/inverse_qho/../utils.py:131: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(arr).float().requires_grad_(g)


In [3]:
dt = (t[1]-t[0])[0]
dx = (x[2]-x[1])[0]

fd_h_t = np.zeros((time_dim, spatial_dim), dtype=np.complex64)
fd_h_x = np.zeros((time_dim, spatial_dim), dtype=np.complex64)
fd_h_xx = np.zeros((time_dim, spatial_dim), dtype=np.complex64)
fd_h_xxx = np.zeros((time_dim, spatial_dim), dtype=np.complex64)

for i in range(spatial_dim):
    fd_h_t[:,i] = FiniteDiff(Exact[:,i], dt, 1)
for i in range(time_dim):
    fd_h_x[i,:] = FiniteDiff(Exact[i,:], dx, 1)
    fd_h_xx[i,:] = FiniteDiff(Exact[i,:], dx, 2)
    fd_h_xxx[i,:] = FiniteDiff(Exact[i,:], dx, 3)

fd_h_t = to_column_vector(fd_h_t)
fd_h_x = to_column_vector(fd_h_x)
fd_h_xx = to_column_vector(fd_h_xx)
fd_h_xxx = to_column_vector(fd_h_xxx)
V = to_column_vector(potential)

In [4]:
derivatives = cat_numpy(h_star.detach().numpy(), V, fd_h_x, fd_h_xx, fd_h_xxx)
dictionary = {}
for i in range(len(feature_names)): dictionary[feature_names[i]] = get_feature(derivatives, i)

In [5]:
c_poly = ComplexPolynomialFeatures(feature_names, dictionary)
complex_poly_features = c_poly.fit()
complex_poly_features

Computing hf
Computing x
Computing h_x
Computing h_xx
Computing h_xxx
Computing hf^2
Computing hf x
Computing hf h_x
Computing hf h_xx
Computing hf h_xxx
Computing x^2
Computing x h_x
Computing x h_xx
Computing x h_xxx
Computing h_x^2
Computing h_x h_xx
Computing h_x h_xxx
Computing h_xx^2
Computing h_xx h_xxx
Computing h_xxx^2


array([[ 1.00000000e+00+0.00000000e+00j,  5.30206137e-07+0.00000000e+00j,
         2.81250000e+01+0.00000000e+00j, ...,
         3.44385238e-11+0.00000000e+00j,  1.08000064e-10+0.00000000e+00j,
         3.38690880e-10+0.00000000e+00j],
       [ 1.00000000e+00+0.00000000e+00j,  5.85638986e-07+0.00000000e+00j,
         2.79057026e+01+0.00000000e+00j, ...,
         4.21659467e-11+0.00000000e+00j,  1.30551893e-10+0.00000000e+00j,
         4.04207614e-10+0.00000000e+00j],
       [ 1.00000000e+00+0.00000000e+00j,  6.46645333e-07+0.00000000e+00j,
         2.76872635e+01+0.00000000e+00j, ...,
         5.06748612e-11+0.00000000e+00j,  1.58847336e-10+0.00000000e+00j,
         4.97928865e-10+0.00000000e+00j],
       ...,
       [ 1.00000000e+00+0.00000000e+00j,  4.36222035e-05+1.03848826e-04j,
         2.74696827e+01+0.00000000e+00j, ...,
        -3.22079218e-05+2.01327684e-04j, -4.89529658e-03-4.40291727e-03j,
         2.05224270e-01-5.55715690e-02j],
       [ 1.00000000e+00+0.00000000e+00j,  3.

In [6]:
w = TrainSTRidge(complex_poly_features, fd_h_t, 1e-10, 10)
print("PDE derived using STRidge")
print_pde(w, c_poly.poly_feature_names)

PDE derived using STRidge
u_t = (0.000006 +0.498821i)h_xx
    + (0.000012 -0.997386i)hf x
   
